In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
dtype_dict = {
    'id_mutation': 'str',
    'date_mutation': 'str',
    'numero_disposition': 'str',
    'nature_mutation': 'str',
    'valeur_fonciere': 'float64',
    'adresse_numero': 'float64',
    'adresse_suffixe': 'str',
    'adresse_nom_voie': 'str',
    'adresse_code_voie': 'str',
    'code_postal': 'str',
    'code_commune': 'str',
    'nom_commune': 'str',
    'code_departement': 'str',
    'ancien_code_commune': 'str',
    'ancien_nom_commune': 'str',
    'id_parcelle': 'str',
    'ancien_id_parcelle': 'str',
    'numero_volume': 'str',
    'lot1_numero': 'str',
    'lot1_surface_carrez': 'float64',
    'lot2_numero': 'str',
    'lot2_surface_carrez': 'float64',
    'lot3_numero': 'str',
    'lot3_surface_carrez': 'float64',
    'lot4_numero': 'str',
    'lot4_surface_carrez': 'float64',
    'lot5_numero': 'str',
    'lot5_surface_carrez': 'float64',
    'nombre_lots': 'int32',
    'code_type_local': 'str',
    'type_local': 'str',
    'surface_reelle_bati': 'float64',
    'nombre_pieces_principales': 'float64',
    'code_nature_culture': 'str',
    'nature_culture': 'str',
    'code_nature_culture_speciale': 'str',
    'nature_culture_speciale': 'str',
    'surface_terrain': 'float64',
    'longitude': 'float64',
    'latitude': 'float64'
}

In [17]:
data = pd.read_csv("data/full_2022.csv",dtype=dtype_dict)
data = data.dropna(subset=['longitude', 'latitude'])

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4571617 entries, 0 to 4617589
Data columns (total 40 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id_mutation                   object 
 1   date_mutation                 object 
 2   numero_disposition            object 
 3   nature_mutation               object 
 4   valeur_fonciere               float64
 5   adresse_numero                float64
 6   adresse_suffixe               object 
 7   adresse_nom_voie              object 
 8   adresse_code_voie             object 
 9   code_postal                   object 
 10  code_commune                  object 
 11  nom_commune                   object 
 12  code_departement              object 
 13  ancien_code_commune           object 
 14  ancien_nom_commune            object 
 15  id_parcelle                   object 
 16  ancien_id_parcelle            object 
 17  numero_volume                 object 
 18  lot1_numero                

In [19]:
data["nature_mutation"].unique()

array(['Vente', "Vente en l'état futur d'achèvement", 'Echange',
       'Vente terrain à bâtir', 'Adjudication', 'Expropriation'],
      dtype=object)

In [20]:
sampled_data = data
sampled_data = sampled_data[sampled_data["nature_mutation"].isin(["Vente","Vente en l'état futur d'achèvement"])]
sampled_data = sampled_data[(sampled_data["nombre_lots"] == 0) | (sampled_data["nombre_lots"] == 1)]
sampled_data = sampled_data[(sampled_data['surface_reelle_bati'].notna()) & (sampled_data['surface_reelle_bati'] != 0)]
sampled_data = sampled_data[(sampled_data['valeur_fonciere'].notna()) & (sampled_data['valeur_fonciere'] != 0)]
sampled_data['code_departement'] = sampled_data['code_departement'].replace(['2A', '2B'], '02')


In [21]:
sampled_data = sampled_data.dropna(subset=['valeur_fonciere'])
columns_to_drop = ["lot1_numero", "lot2_numero", "lot3_numero", "lot4_numero", "lot5_numero","ancien_code_commune",
                   "ancien_nom_commune","adresse_numero","adresse_suffixe","id_parcelle","ancien_id_parcelle","code_type_local",
                   "numero_volume","code_nature_culture_speciale","nature_culture_speciale","code_nature_culture",
                   "code_nature_culture","numero_disposition","id_mutation","lot2_surface_carrez","lot3_surface_carrez","lot4_surface_carrez",
                   "lot5_surface_carrez","adresse_nom_voie","adresse_code_voie","code_postal","code_commune","nom_commune","nombre_lots",
                   "lot1_surface_carrez","nature_culture"]
    
sampled_data = sampled_data.drop(columns=columns_to_drop)

In [22]:
import pickle
import os

def save_obj(obj,name):
            with open( name + '.pkl', 'wb') as f:
                pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
 
for dirname, _, filenames in os.walk('data\BallTree'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [23]:
sampled_data['latitude'] = sampled_data['latitude'].astype(float)
sampled_data['longitude'] = sampled_data['longitude'].astype(float)


# Fill NA values with 0 for 'surface_terrain'
sampled_data["surface_terrain"] = sampled_data["surface_terrain"].fillna(0)

# Convert 'valeur_fonciere' to string, replace commas with periods, and then convert to float
sampled_data["valeur_fonciere"] = sampled_data["valeur_fonciere"].astype(str).apply(lambda x: x.replace(",", ".")).astype(float)

# Ensure that the conversion to integer happens after handling NaN or other non-numeric values
# sampled_data["valeur_fonciere"] = sampled_data["valeur_fonciere"].astype(int)

# Calculate 'Prix_m' and round it
sampled_data["Prix_m"] = sampled_data["valeur_fonciere"] / sampled_data["surface_reelle_bati"]
sampled_data["Prix_m"] = sampled_data["Prix_m"].apply(lambda x: round(x))

sampled_data['latitude_r'] = np.deg2rad(sampled_data['latitude'])
sampled_data['longitude_r'] = np.deg2rad(sampled_data['longitude'])
sampled_data=sampled_data[sampled_data.latitude.notna()]
sampled_data=sampled_data[sampled_data.code_departement.notna()]

sampled_data=sampled_data.sort_values(by = ['latitude','longitude'])
sampled_data=sampled_data.drop(columns=['latitude','longitude'])
sampled_data=sampled_data.reset_index(drop=True)


In [24]:
sampled_data.pivot_table(values='valeur_fonciere', index='type_local', columns='nature_mutation', aggfunc='size')

nature_mutation,Vente,Vente en l'état futur d'achèvement
type_local,,
Appartement,413920,8455
Local industriel. commercial ou assimilé,113165,1952
Maison,740502,547


In [25]:
appart_old=sampled_data[(sampled_data['type_local']=='Appartement') & (sampled_data['nature_mutation']=='Vente')]
appart_old.reset_index(drop=True,inplace=True)
maison_old=sampled_data[(sampled_data['type_local']=='Maison') & (sampled_data['nature_mutation']=='Vente')]
maison_old=maison_old[(maison_old['valeur_fonciere']>20000)&(maison_old['Prix_m']>100)&(maison_old['surface_reelle_bati']>9)]
maison_old.reset_index(drop=True,inplace=True)
local_old =sampled_data[(sampled_data['type_local']=='Local industriel. commercial ou assimilé') & (sampled_data['nature_mutation']=='Vente')]
local_old.reset_index(drop=True,inplace=True)
appart_new=sampled_data[(sampled_data['type_local']=='Appartement') & (sampled_data['nature_mutation']=='Vente en l\'état futur d\'achèvement')]
appart_new.reset_index(drop=True,inplace=True)

In [26]:
maison_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728802 entries, 0 to 728801
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   date_mutation              728802 non-null  object 
 1   nature_mutation            728802 non-null  object 
 2   valeur_fonciere            728802 non-null  float64
 3   code_departement           728802 non-null  object 
 4   type_local                 728802 non-null  object 
 5   surface_reelle_bati        728802 non-null  float64
 6   nombre_pieces_principales  728802 non-null  float64
 7   surface_terrain            728802 non-null  float64
 8   Prix_m                     728802 non-null  int64  
 9   latitude_r                 728802 non-null  float64
 10  longitude_r                728802 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 61.2+ MB


## ALL ##

In [27]:
from sklearn.neighbors import BallTree
import os
import pickle

# Assurez-vous que vos données sont correctement chargées et préparées
# appart_old, maison_old, local_old, appart_new doivent être définis et préparés avant ce point

def save_obj(obj, name):
    # Créez le répertoire s'il n'existe pas
    directory = "data/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Sauvegardez l'objet
    with open(os.path.join(directory, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# Initialiser les modèles
models = {}

# Fusionnez les trois types de biens dans un DataFrame unique
all_properties = pd.concat([appart_old, maison_old, local_old,appart_new])

# Calculez la distance moyenne et les indices des voisins (si nécessaire)
all_properties['distance_moyenne'] = np.zeros(len(all_properties))
all_properties['indices_voisins'] = np.zeros(len(all_properties))

# Créez un modèle BallTree pour toute la France
model_all = BallTree(all_properties[['latitude_r', 'longitude_r']].values, leaf_size=2, metric='haversine')
models['france_entiere'] = model_all

# Sauvegardez le modèle
save_obj(models['france_entiere'], 'france_entiere')


In [28]:
all_properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1264342 entries, 0 to 8454
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   date_mutation              1264342 non-null  object 
 1   nature_mutation            1264342 non-null  object 
 2   valeur_fonciere            1264342 non-null  float64
 3   code_departement           1264342 non-null  object 
 4   type_local                 1264342 non-null  object 
 5   surface_reelle_bati        1264342 non-null  float64
 6   nombre_pieces_principales  1264342 non-null  float64
 7   surface_terrain            1264342 non-null  float64
 8   Prix_m                     1264342 non-null  int64  
 9   latitude_r                 1264342 non-null  float64
 10  longitude_r                1264342 non-null  float64
 11  distance_moyenne           1264342 non-null  float64
 12  indices_voisins            1264342 non-null  float64
dtypes: float64(8), int64

In [29]:
import time
import pandas as pd
import os
from sklearn.neighbors import BallTree

# Assurez-vous que vos données sont correctement chargées et préparées
# appart_old, maison_old, local_old doivent être définis et préparés avant ce point

# Créez un répertoire pour sauvegarder les données combinées
directory = "data/france_entiere"
if not os.path.exists(directory):
    os.makedirs(directory)

# Fusionnez les trois types de biens dans un DataFrame unique
all_properties = pd.concat([appart_old, maison_old, local_old])

# Créez un modèle BallTree pour toute la France
model_all = BallTree(all_properties[['latitude_r', 'longitude_r']].values, leaf_size=2, metric='haversine')

# Initialiser une liste pour stocker les données de toutes les propriétés
all_data = []

# Itérez sur chaque type de bien
for property_type, data_old in [('appart', appart_old), ('maison', maison_old), ('local', local_old)]:
    start = time.time()

    # Vérifiez si le DataFrame a au moins 10 lignes
    k_nearest = min(10, len(data_old))

    # Calculez la distance et les indices pour les k biens les plus proches
    dist, indices = model_all.query(data_old[['latitude_r', 'longitude_r']].values, k=k_nearest)

    # Calculez la distance moyenne
    data_old['distance_moyenne'] = np.mean(dist[:, 1:] * 6341, axis=1)  # Assurez-vous que la conversion est correcte

    # Initialisez le DataFrame pour le prix moyen du quartier
    prix_moyen_quartier = pd.DataFrame({'Prix_m': np.zeros(len(data_old))})

    # Boucle pour calculer le prix moyen
    for i in range(1, k_nearest):
        # Gérer les indices hors limites
        valid_indices = indices[:, i] < len(data_old)
        prix_valides = pd.DataFrame(data_old.iloc[indices[valid_indices, i]]['Prix_m']).reset_index(drop=True)
        prix_moyen_quartier.loc[valid_indices, 'Prix_m'] += prix_valides['Prix_m']

    prix_moyen_quartier /= (k_nearest - 1)
    data_old['Prix_moyen_du_quartier'] = prix_moyen_quartier.values

    stop = time.time()
    print(f"Type: {property_type}, Time taken: {stop - start} seconds")

    # Ajoutez les données traitées à la liste
    all_data.append(data_old)

# Concaténez toutes les dataframes dans la liste
combined_data = pd.concat(all_data)

# Sauvegardez les données combinées dans un seul fichier CSV
combined_csv_filename = os.path.join(directory, "combined_properties_france.csv")
combined_data.to_csv(combined_csv_filename, index=False, header=True)


C:\Users\arthu\AppData\Local\Temp\ipykernel_26944\931870615.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_old['distance_moyenne'] = np.mean(dist[:, 1:] * 6341, axis=1)  # Assurez-vous que la conversion est correcte
C:\Users\arthu\AppData\Local\Temp\ipykernel_26944\931870615.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_old['Prix_moyen_du_quartier'] = prix_moyen_quartier.values


Type: appart, Time taken: 81.48812484741211 seconds
Type: maison, Time taken: 128.90504431724548 seconds


C:\Users\arthu\AppData\Local\Temp\ipykernel_26944\931870615.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_old['distance_moyenne'] = np.mean(dist[:, 1:] * 6341, axis=1)  # Assurez-vous que la conversion est correcte
C:\Users\arthu\AppData\Local\Temp\ipykernel_26944\931870615.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_old['Prix_moyen_du_quartier'] = prix_moyen_quartier.values


Type: local, Time taken: 21.774197578430176 seconds


## graph ##

In [30]:
all = pd.read_csv("data/france_entiere/combined_properties_france.csv")


In [31]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255887 entries, 0 to 1255886
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   date_mutation              1255887 non-null  object 
 1   nature_mutation            1255887 non-null  object 
 2   valeur_fonciere            1255887 non-null  float64
 3   code_departement           1255887 non-null  int64  
 4   type_local                 1255887 non-null  object 
 5   surface_reelle_bati        1255887 non-null  float64
 6   nombre_pieces_principales  1255887 non-null  float64
 7   surface_terrain            1255887 non-null  float64
 8   Prix_m                     1255887 non-null  int64  
 9   latitude_r                 1255887 non-null  float64
 10  longitude_r                1255887 non-null  float64
 11  distance_moyenne           1255887 non-null  float64
 12  Prix_moyen_du_quartier     999075 non-null   float64
dtypes: float64(8

In [32]:
all[all['Prix_moyen_du_quartier'].isna()]


,date_mutation,nature_mutation,valeur_fonciere,code_departement,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,Prix_m,latitude_r,longitude_r,distance_moyenne,Prix_moyen_du_quartier
289175,2022-07-13,Vente,133000.0,77,Appartement,59.0,2.0,0.0,2254,0.847310,0.046264,0.082111,NaN
289176,2022-07-15,Vente,160000.0,77,Appartement,60.0,3.0,0.0,2667,0.847310,0.046264,0.082111,NaN
289177,2022-03-28,Vente,85000.0,61,Appartement,40.0,3.0,0.0,2125,0.847310,-0.007394,0.119132,NaN
289178,2022-11-24,Vente,88000.0,61,Appartement,45.0,3.0,0.0,1956,0.847310,-0.007394,0.119132,NaN
289186,2022-02-11,Vente,135000.0,77,Appartement,45.0,2.0,0.0,3000,0.847316,0.046259,0.085987,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163303,2022-06-27,Vente,157000.0,30,Local industriel. commercial ou assimilé,60.0,0.0,0.0,2617,0.764841,0.075731,0.157546,NaN
1163313,2022-07-08,Vente,800000.0,30,Local industriel. commercial ou assimilé,142.0,0.0,0.0,5634,0.764843,0.074516,0.087972,NaN
1163321,2022-12-09,Vente,660000.0,84,Local industriel. commercial ou assimilé,145.0,0.0,105.0,4552,0.764856,0.092609,0.118166,NaN
1163375,2022-10-19,Vente,870000.0,84,Local industriel. commercial ou assimilé,120.0,0.0,83.0,7250,0.764864,0.092639,0.068027,NaN
